In [1]:
# Initial imports.
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.datasets import make_classification

In [2]:
# Loading data
# file_path = 
# df = pd.read_csv(file_path)
# df.head()

# Define the target set.
# target = 'Target_Column_Name'
# y = df[target].values()

# Define the features set.
# X = df.copy()
# X = X.drop(columns=target)

### ---------------------PLACEHOLDER----------------------
### The final csv will be uploaded to github, therefore the raw csv will be accessed using the github url
### This is a test dataframe
X, y = make_classification(random_state=18, n_classes=2, n_samples=400, n_features=300)

In [3]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=18)

In [4]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [5]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=256, random_state=18) 

In [6]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [7]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [8]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,44,10
Actual 1,10,36


In [9]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,44,10
Actual 1,10,36


Accuracy Score : 0.8
Classification Report
              precision    recall  f1-score   support

           0       0.81      0.81      0.81        54
           1       0.78      0.78      0.78        46

    accuracy                           0.80       100
   macro avg       0.80      0.80      0.80       100
weighted avg       0.80      0.80      0.80       100



In [10]:
# Sort the features by their importance.
sorted(zip(rf_model.feature_importances_, pd.DataFrame(data=X).columns), reverse=True)

[(0.14516057149214404, 25),
 (0.09423072727120863, 205),
 (0.07032040032761255, 282),
 (0.007556180942417702, 259),
 (0.007026122945758891, 128),
 (0.005757419325489492, 216),
 (0.005586291996339313, 43),
 (0.0055412677412443895, 13),
 (0.005130014376873578, 281),
 (0.0050961181998928, 250),
 (0.005019295694505154, 155),
 (0.004714783325077108, 111),
 (0.004668383319135028, 145),
 (0.004256481793775319, 174),
 (0.004195434127353672, 217),
 (0.004157572377146531, 195),
 (0.003952828336866818, 38),
 (0.0037087890187965937, 71),
 (0.003692914639097322, 122),
 (0.0036898675397970884, 100),
 (0.0036488040761481666, 244),
 (0.003631830512094279, 232),
 (0.003602644702490096, 101),
 (0.00357618512800711, 65),
 (0.0035535950586738103, 108),
 (0.0034823351855142438, 99),
 (0.003445830216702478, 66),
 (0.0034457609356978807, 102),
 (0.0034447368118438273, 161),
 (0.0033969429791985846, 18),
 (0.0033966930501462037, 277),
 (0.0033841816932781606, 251),
 (0.003319781452881159, 49),
 (0.00327713544